In [1]:

######## snakemake preamble start (automatically inserted, do not edit) ########
library(methods)
Snakemake <- setClass(
    "Snakemake",
    slots = c(
        input = "list",
        output = "list",
        params = "list",
        wildcards = "list",
        threads = "numeric",
        log = "list",
        resources = "list",
        config = "list",
        rule = "character",
        bench_iteration = "numeric",
        scriptdir = "character",
        source = "function"
    )
)
snakemake <- Snakemake(
    input = list('data/ATACseq_250kb_bins.txt', 'data/cases_controls/cases_controls_rds_format/all_samples_normalized_gaussian_smooth_trimmed_CONTROLS.rds', "input_atac" = 'data/ATACseq_250kb_bins.txt', "input_train_data" = 'data/cases_controls/cases_controls_rds_format/all_samples_normalized_gaussian_smooth_trimmed_CONTROLS.rds'),
    output = list('data/ATAC_predictions_new_split/train_gaussian_smooth_trimmed_CONTROLS_split.rds', 'data/ATAC_predictions_new_split/test_gaussian_smooth_trimmed_CONTROLS_split.rds', "train_data_output" = 'data/ATAC_predictions_new_split/train_gaussian_smooth_trimmed_CONTROLS_split.rds', "test_data_output" = 'data/ATAC_predictions_new_split/test_gaussian_smooth_trimmed_CONTROLS_split.rds'),
    params = list(),
    wildcards = list('gaussian_smooth_trimmed_CONTROLS', "suffix" = 'gaussian_smooth_trimmed_CONTROLS'),
    threads = 1,
    log = list('logs/processed_notebooks/experiments/splitting_80_to_smaller_gaussian_smooth_trimmed_CONTROLS.r.ipynb', "notebook" = 'logs/processed_notebooks/experiments/splitting_80_to_smaller_gaussian_smooth_trimmed_CONTROLS.r.ipynb'),
    resources = list('mem_mb', 'disk_mb', 'tmpdir', 'time', "mem_mb" = 200000, "disk_mb" = 10530, "tmpdir" = '/scratch/56738039', "time" = '48:00:00'),
    config = list(),
    rule = 'splitting_final',
    bench_iteration = as.numeric(NA),
    scriptdir = '/faststorage/project/DELFI1/Workspaces/CarmenAndAnika',
    source = function(...){
        wd <- getwd()
        setwd(snakemake@scriptdir)
        source(...)
        setwd(wd)
    }
)
setwd('/faststorage/project/DELFI1/Workspaces/CarmenAndAnika');

######## snakemake preamble end #########


In [2]:
library(tidyverse)

Registered S3 methods overwritten by 'ggplot2':
  method         from 
  [.quosures     rlang
  c.quosures     rlang
  print.quosures rlang
Registered S3 method overwritten by 'rvest':
  method            from
  read_xml.response xml2
── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──
✔ ggplot2 3.1.1       ✔ purrr   0.3.2  
✔ tibble  2.1.1       ✔ dplyr   0.8.0.1
✔ tidyr   0.8.3       ✔ stringr 1.4.0  
✔ readr   1.3.1       ✔ forcats 0.4.0  
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


In [3]:

INPUT_percentage = 0.2

In [4]:
INPUT_std_LASSO_TRAIN_DATA = readRDS(snakemake@input[["input_train_data"]])
#INPUT_std_LASSO_TRAIN_DATA = readRDS("data/cases_controls/cases_controls_rds_format/all_samples_normalized_CONTROLS.rds")

In [5]:
percentage = INPUT_percentage

In [6]:
n_input = round((dim(INPUT_std_LASSO_TRAIN_DATA)[1]/10303)*percentage)

In [7]:
(n_input = as.integer(n_input))

[1] 49

In [8]:
INPUT_std_LASSO_TRAIN_DATA %>%             
  summarise(Unique_Elements = n_distinct(sample))

ind_list = INPUT_std_LASSO_TRAIN_DATA %>% distinct(sample)
ind_list = ind_list$sample
length(ind_list)


set.seed(0)
training <- sample(ind_list, size = n_input, replace = FALSE)

std_LASSO_test <- INPUT_std_LASSO_TRAIN_DATA %>% filter(!sample %in% training)
std_LASSO_train <- INPUT_std_LASSO_TRAIN_DATA %>% filter(sample %in% training)


Unique_Elements
243


[1] 243

In [9]:
saveRDS(std_LASSO_test, file = snakemake@output[["test_data_output"]])
saveRDS(std_LASSO_train, file = snakemake@output[["train_data_output"]])
#saveRDS(std_LASSO_train_20, file = snakemake@output[["OUTPUT_norm_trimmed_LASSO_DATA_20"]])